# Forecasting Weekly Sales Across Walmart Stores Using Prophet

## The Problem at Hand

The purpose of this project is to forecast future sales predictions for forty five different Walmart stores in the USA and to determine if additional features may prove most useful to support this. Why is forecasting useful? Forecasting can provide insights to aid logistical management of stores for example, stock and staff planning to ensure stock meets demand whilst ensuring the store is neither under or over staffed to maintain cost effectiveness. 

The dataset used for this project comes from Kaggle authored by M Yasser H and can be found by following this link: [Walmart Dataset](https://www.kaggle.com/datasets/yasserh/walmart-dataset). The time span of this data ranges from 5/2/2010 to 1/11/2012 where sales data is represented weekly. Additional variables within this dataset include: Store (Store number), Holiday_Flag (Binary, 1 if holiday occured during a week), Temperature (Fahrenheit), Fuel_Price(US$ per gallon), CPI (Consumer price index)and Unemployment (Unemployment rate). 


In [ ]:
import os
os.chdir(r"C:\Users\tjsla\OneDrive\Desktop\Personal projects\Walmart sales forecasting\scripts_functions")
import pandas as pd
import seaborn as sns
from dcf_test_1 import check_zeros_nas
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import plotly.express as px
import plotly.io as pio
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from prophet import Prophet
import seaborn as sns

## Exploratory Data Analysis

In [ ]:
Full_Walmart = pd.read_csv("C:/Users/tjsla/OneDrive/Desktop/Personal projects/Walmart sales forecasting/Data/Walmart.csv")
Full_Walmart["Date"] = pd.to_datetime(Full_Walmart["Date"],format="%d-%m-%Y")

stores = Full_Walmart['Store'].unique()
store_dfs = {store: Full_Walmart[Full_Walmart['Store'] == store][['Date', 'Weekly_Sales']] for store in stores}

for store in stores:
    df = store_dfs[store].copy()
    df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y', 'cap': 'cap'})
    store_dfs[store] = df

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

store_errors = {}

for store in stores:
    df = store_dfs[store].copy()
    df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})
    df['ds'] = pd.to_datetime(df['ds'], dayfirst=True, errors='coerce') 
    df = df.dropna(subset=['ds', 'y']) 
    store_dfs[store] = df

combined_df = pd.concat(store_dfs.values(), keys=store_dfs.keys(), names=['Store'])
combined_df = combined_df.reset_index()

plt.figure(figsize=(12,5))
sns.boxplot(x='Store', y='y', data=combined_df)
plt.title('Weekly Sales Distribution per Store')
plt.ylabel('Weekly Sales (Millions)')
plt.show()

<p><b>Figure 1:</b> Boxplot of weekly sale ranges for each store.</p>


The initial exploration involved understanding the sales distributions for each store to determine if there are any patterns or unique cases. Figure 1 provides information on the sale ranges for all 45 stores where, a key detail is the presence of outliers for many stores typically representing high sale volumes. Usually the reasons for higher sales stem from a number of factors such as promotions and holiday periods, in this case the data only provided information for holiday events.

In [ ]:
import plotly.graph_objects as go

Full_Walmart['Year'] = Full_Walmart['Date'].dt.year

sales_by_holiday_year = (
    Full_Walmart.groupby(['Holiday_Flag', 'Year'])['Weekly_Sales']
    .sum()
    .reset_index()
)

sales_by_holiday_year['Holiday'] = sales_by_holiday_year['Holiday_Flag'].map({0: 'Non-Holiday', 1: 'Holiday'})

pivot_df = sales_by_holiday_year.pivot(index='Year', columns='Holiday', values='Weekly_Sales')
percent_df = pivot_df.div(pivot_df.sum(axis=1), axis=0) * 100

fig = go.Figure()

for label, color in zip(['Non-Holiday', 'Holiday'], ['skyblue', 'green']):
    fig.add_trace(
        go.Bar(
            x=pivot_df.index.astype(str),
            y=pivot_df[label],
            name=label,
            marker_color=color,
            customdata=percent_df[label].values.reshape(-1, 1),
            hovertemplate=(
                "<b>" + label + "</b><br>" +
                "Year: %{x}<br>" +
                "Sales: %{y:,}<br>" +
                "Percent: %{customdata[0]:.1f}%<extra></extra>"
            )
        )
    )


fig.update_layout(
    barmode='stack',
    title='Total Weekly Sales by Year: Holiday vs Non-Holiday',
    xaxis_title='Year',
    yaxis_title='Total Sales',
    legend_title='Week Type',
    template='plotly_white',
    height=600,
    width = 600
)

fig.show()

<p><b>Figure 2:</b> Stacked plot of total sales across all stores by year for holiday and non-holiday weeks (Interactable).</p>

Figure 2 provides insight into the total sales for each year and the proportion of which occurred during holiday periods. For each year the percentage of sales which fell into holiday weeks were as follows: [8.7%],[8.4%] and [4.9%]. Although 2012 experienced the least number of total and holiday sales, this is likely explained by data ending on 26/10/2012 where notably the Christmas period is missing for this year which may contribute to the lower values.

In [ ]:
store_totals = Full_Walmart.groupby("Store")["Weekly_Sales"].sum().reset_index()
store_totals.columns = ["Store", "Total_Sales"]

store_totals["Performance_Tier"] = pd.qcut(
    store_totals["Total_Sales"], 
    q=3,
    labels=["Low", "Medium", "High"]
)
Full_Walmart = Full_Walmart.merge(store_totals[["Store", "Performance_Tier"]], on="Store")

high_perf = Full_Walmart[Full_Walmart["Performance_Tier"] == "High"]
medium_perf = Full_Walmart[Full_Walmart["Performance_Tier"] == "Medium"]
low_perf = Full_Walmart[Full_Walmart["Performance_Tier"] == "Low"]

bright_13_palette = [
    "#e6194b",  # red
    "#3cb44b",  # green
    "#ffe119",  # yellow
    "#4363d8",  # blue
    "#f58231",  # orange
    "#911eb4",  # purple
    "#46f0f0",  # cyan
    "#f032e6",  # magenta
    "#bcf60c",  # lime
    "#fabebe",  # pink
    "#008080",  # teal
    "#e6beff",  # lavender
]
yticks_hp = np.arange(1000000, 4000000 + 1, 500000)

hp_plotly = px.line(
    high_perf,
    x="Date",
    y="Weekly_Sales",
    color="Store",
    title="High Performance Store Sales",
    labels={
        "Date": "Date",
        "Weekly_Sales": "Weekly Sales",
        "Store": "Store Number"
    },
    color_discrete_sequence=bright_13_palette, 
    template = "plotly_white"
)

hp_plotly.update_yaxes(tickformat=",", title="Weekly Sales")

hp_plotly.update_xaxes(
    dtick="M1", 
    tickformat="%b\n%Y",
    title="Date"
)

hp_plotly.update_yaxes(
    tickformat=",",
    tickvals=yticks_hp,
    title="Weekly Sales"
)
hp_plotly.update_traces(line=dict(width=1))


hp_plotly.show()

<p><b>Figure 3:</b> Lineplot of weekly sales for upper 33% performance stores (Interactable).</p>

In [ ]:
yticks_mp = np.arange(500000, 2500000 + 1, 250000)

mp_plotly = px.line(
    medium_perf,
    x="Date",
    y="Weekly_Sales",
    color="Store",
    title="Medium Performance Store Sales",
    labels={
        "Date": "Date",
        "Weekly_Sales": "Weekly Sales",
        "Store": "Store Number"
    },
    color_discrete_sequence=bright_13_palette,  
    template = "plotly_white"
)

mp_plotly.update_yaxes(tickformat=",", title="Weekly Sales")


mp_plotly.update_xaxes(
    dtick="M1",  
    tickformat="%b\n%Y",  
    title="Date"
)

mp_plotly.update_yaxes(
    tickformat=",",
    tickvals=yticks_mp,
    title="Weekly Sales"
)
mp_plotly.update_traces(line=dict(width=1))

mp_plotly.show()

<p><b>Figure 4:</b> Lineplot of weekly sales for middle 33% performance stores (Interactable).</p>

In [ ]:
yticks_lp = np.arange(200000, 1400000 + 1, 200000)

lp_plotly = px.line(
    low_perf,
    x="Date",
    y="Weekly_Sales",
    color="Store",
    title="Low Performance Store Sales",
    labels={
        "Date": "Date",
        "Weekly_Sales": "Weekly Sales",
        "Store": "Store Number"
    },
    color_discrete_sequence=bright_13_palette,  
    template = "plotly_white"
)

lp_plotly.update_yaxes(tickformat=",", title="Weekly Sales")

lp_plotly.update_xaxes(
    dtick="M1", 
    tickformat="%b\n%Y",  
    title="Date"
)

lp_plotly.update_yaxes(
    tickformat=",",
    tickvals=yticks_lp,
    title="Weekly Sales"
)
lp_plotly.update_traces(line=dict(width=1))

lp_plotly.show()

<p><b>Figure 5:</b> Lineplot of weekly sales for lower 33% performance stores (Interactable).</p>

Figures 3-5 provide insight into store sale trends over time where stores had been divided into three groups based on a three quartile split. High performing stores consist of the top 33% performing stores based on sales, medium performance the middle 33% and low performance on the lowest 33% of stores. Figure 3 implied top performing stores typically shared similar patterns with major peaks around Christmas. Similarly medium performance stores also follow the same trend however, stores 28 and 41 notably performed better than other stores during non Christmas periods suggesting they may have been better grouped within the high performance stores. Additionally, store 18 saw decreased sales during September 2011 later recovering in October/November implying an event occurred during this period however, the cause is unknown. Finally, low performance stores also typically share the same pattern with some exceptions. For example, store 33 notably does not see as dramatic an increase in sales during the Christmas period with sales remaining roughly stable throughtout the time period comparatively, store 36 implied a decrease in sales overtime.

## Feature Importance
Following the exploratory analysis, although prophet is a time series based model approach and therefore does not typically use additional features, feature importance was still utilised to determine if any additional features would improve model accuracy.

In [ ]:
numerical_features = ['Weekly_Sales', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment']
corr = Full_Walmart[numerical_features].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

<p><b>Figure 6:</b> Correlation matrix of numerical features and their relationship with sales.</p>
Figure 6 implied little relationship between the features available with the strongest relationship being between CPI and Unemployment [-0.30] which implied a minor negative relationship. This implied that no features present had a strong relationship with weekly sales suggesting they may be unnecessary for the prophet model. 

In [ ]:
features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Holiday_Flag']
X = Full_Walmart[features]
y = Full_Walmart['Weekly_Sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xgb = XGBRegressor(random_state=42,enable_categorical=True)
xgb.fit(X_train, y_train)

import shap

explainer = shap.TreeExplainer(xgb)
shap_values = explainer(X_test)

shap.summary_plot(shap_values, X_test,show=False)
ax = plt.gca()

max_val = ax.get_xlim()[1]
xticks = np.arange(-900000, max_val + 1, 300000)
ax.set_xticks(xticks)
plt.tight_layout()
plt.show()

p><b>Figure 7:</b> Beeswarm plot of SHAP values for additional features.</p>
Additionally, figure 7 provides further insight into feature impact on weekly sales. In this case feature effects were evaluated using an XGBoost model to invetsigate effects on model accuracy. The SHAP values indicated that unemployment and CPI generally had the greatest impact on weekly sales whilst holiday flags had minimal impact with higher values potentially being related to the Christmas periods specifically. However, it should be noted that the direction of each variable is somewhat ambigous although, unemployment and CPI generally suggested higher values would negatively impact sales. Overall, due to a lack of clear contributions no additional features were implemented into the prophet model to avoid reductions in accuracy.

## Methodology
In order to prepare the data for modelling the first step involved checking for missing values. This was conducted using a function called check_zeros_nas which totalled NA and 0 value recordings for each column and storing the results. The output of this function implied no missing values or unexpected 0 recordings were present. Following this the "date" variable was formatted to ensure it was in the correct date-time format before being passed to prophet. 

As mentioned additional features were tested using an XGBoost model. This was conducted by running the XGBoost regression model using all available features and then applying the results to both permutation importance and shap algorithms. 

Before conducting forecasting a cross validation setup (prophet uses rolling-origin evaluation) was applied to the initial model using a setup of an initial 730 days, 90 day period and 90 day horizon. This cross validation was used to evaluate how well the model forecasts future data alongside tuning the changepoint paramater.

Forecasting was then conducted using the same setup as it proved effective, alongside a changepoint value of 0.5 being utilised.

## Forecasting Results

In [ ]:
import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)

stores = Full_Walmart['Store'].unique()
store_dfs = {store: Full_Walmart[Full_Walmart['Store'] == store][['Date', 'Weekly_Sales']] for store in stores}

for store in stores:
    df = store_dfs[store].copy()
    df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y', 'cap': 'cap'})
    store_dfs[store] = df

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

store_errors = {}

for store in stores:
    df = store_dfs[store].copy()
    df = df.rename(columns={'Date': 'ds', 'Weekly_Sales': 'y'})
    df['ds'] = pd.to_datetime(df['ds'], dayfirst=True, errors='coerce')  
    df = df.dropna(subset=['ds', 'y'])  
    store_dfs[store] = df

for store in stores:
    df = store_dfs[store]
    model = Prophet(yearly_seasonality=True, weekly_seasonality=True,\
                    changepoint_prior_scale=0.5)    
    model.fit(df)

    try:
        df_cv = cross_validation(model, initial='730 days', period='90 days', horizon='90 days', parallel="processes")
        df_p = performance_metrics(df_cv)
        mape = df_p['mape'].mean()
        store_errors[store] = mape
    except Exception as e:
        print(f"Store {store}: error during cross-validation - {e}")
        store_errors[store] = None

results_df = pd.DataFrame(list(store_errors.items()), columns=['Store', 'MAPE'])
results_df = results_df.dropna().sort_values('MAPE');


In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(results_df['Store'].astype(str), results_df['MAPE'])
plt.xlabel('Store')
plt.ylabel('Mean Absolute Percentage Error (MAPE)')
plt.title('Prophet Forecasting Accuracy by Store')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

<p><b>Figure 8:</b> Barplot of MAPE scores based on cross validation.</p>

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


mape_values = results_df[['MAPE']].copy()
store_labels = results_df['Store'].astype(str).values

scaler = StandardScaler()
mape_scaled = scaler.fit_transform(mape_values)

kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(mape_scaled)
results_df['Cluster'] = clusters

k_palette = ["#f032e6", "#f58231", "#4363d8"]

cluster_ranges = results_df.groupby('Cluster')['MAPE'].agg(['min', 'max']).sort_values(by='min')

plt.figure(figsize=(10, 5))

for i, (cluster_id, row) in enumerate(cluster_ranges.iterrows()):
    plt.axhspan(
        row['min'], row['max'],
        facecolor=k_palette[cluster_id], 
        alpha=0.1, 
        label=f'Cluster {cluster_id}'
    )

sns.scatterplot(data=results_df, x='Store', y='MAPE', hue='Cluster', palette=k_palette, s=100)

for i in range(results_df.shape[0]):
    plt.text(
        x=results_df['Store'].iloc[i], 
        y=results_df['MAPE'].iloc[i] + 0.002,
        s=str(results_df['Store'].iloc[i]), 
        fontsize=8, 
        ha='center', 
        va='bottom'
    )

plt.xticks(rotation=90)
plt.title('Store Forecasting MAPE Clusters with Background Regions')
plt.ylabel('MAPE')
plt.xlabel('Store')
plt.tight_layout()
plt.legend(title='Cluster')
plt.show()

<p><b>Figure 9:</b> Scatter plot of store forecast performance groupings based on MAPE score.</p>

Before forecasting, cross validation of the prophet model was used to assess performance on unseen data to ensure the model avoided overfitting. Figures 8 and 9 provide the cross validation results where error scores ranged between 0.02 and 0.1 which respectively mean 2% and 10%. Thus implying the forecast error is generally low as where a typical standard is: 0%-10% high accuracy, 11%-20% good accuracy and 21%-30% being acceptable. However, stores within the group 1 cluster may require additional features to improve forecast accuracy due to having the highest amount of error. Forecasting accuracy was based on a 13week (3 months) period into the future, an extened period of forecasting would require more data to maintain accuracy which will be evidenced by the forecast plots. However, overall this implied the forecasts generated by the model are highly accurate evidencing seasonality alone may be enough to explain weekly sales. 

In [ ]:
forecast_horizon = 13  
store_forecasts = {}
store_anomalies = {}

stores = sorted(stores)  

plots_per_fig = 9
rows, cols = 3, 3

for idx, store in enumerate(stores):
    df = store_dfs[store].copy()
    df['ds'] = pd.to_datetime(df['ds'], dayfirst=True, errors='coerce')
    df = df.dropna(subset=['ds', 'y'])

    model = Prophet(
        yearly_seasonality=True,
        weekly_seasonality=True,
        changepoint_prior_scale=0.5
    )
    model.fit(df)

    future = model.make_future_dataframe(periods=forecast_horizon, freq='W')
    forecast = model.predict(future)

    forecast = forecast.merge(df[['ds', 'y']], on='ds', how='left')
    forecast.rename(columns={'y': 'fact'}, inplace=True)
    forecast['anomaly'] = 0
    forecast.loc[forecast['fact'] > forecast['yhat_upper'], 'anomaly'] = 1
    forecast.loc[forecast['fact'] < forecast['yhat_lower'], 'anomaly'] = -1
    forecast['importance'] = 0.0
    forecast.loc[forecast['anomaly'] == 1, 'importance'] = (
        (forecast['fact'] - forecast['yhat_upper']) / forecast['fact']
    )
    forecast.loc[forecast['anomaly'] == -1, 'importance'] = (
        (forecast['yhat_lower'] - forecast['fact']) / forecast['fact']
    )

    store_forecasts[store] = forecast
    store_anomalies[store] = forecast[forecast['anomaly'] != 0]

    if idx % plots_per_fig == 0:
        fig, axes = plt.subplots(rows, cols, figsize=(18, 12))
        axes = axes.flatten()

    ax = axes[idx % plots_per_fig]
    
    ax.plot(df['ds'], df['y'], label='Actual', color='black', linewidth=1)
    ax.plot(forecast['ds'], forecast['yhat'], label='Forecast', color='#f032e6', linewidth=2)
    ax.plot(forecast['ds'][-forecast_horizon:], forecast['yhat'][-forecast_horizon:], label='Forecast Horizon', color='purple', linewidth=3)
    ax.fill_between(forecast['ds'], forecast['yhat_lower'], forecast['yhat_upper'], color='grey', alpha=0.2)

    anomalies = forecast[forecast['anomaly'] != 0]
    ax.scatter(anomalies['ds'], anomalies['fact'], color='red', s=40, label='Anomaly', zorder=5)

    ax.set_title(f'Store {store}', fontsize=10)
    ax.tick_params(axis='x', rotation=30)

    if idx % plots_per_fig == plots_per_fig - 1 or store == stores[-1]:
        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels, loc='upper left', ncol=4)
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        plt.suptitle('Walmart Store Weekly Sale Forecasts with Anomalies', fontsize=16)
        plt.show()

<p><b>Figures 10-14:</b> Lineplots for stores 1-45 showing model uncertainty and forecast prediction whilst identifying potential anomalies.</p>

Figures 10-14 provide the forecast predictions and associated uncertainty for each store where red points indicate potential anomalies. However, skepticism should be employed for anomalous points as they are based on if they fall outside the models expectations. Although, anomalous points that fall far from the expected range are more probable to be correctly identified as many such points occur during the Christmas period where an anticipated dramatic increase in sales is expected. Due to a lack of data other extremes are more difficult to interpret and could correlate to a number of events such as other holiday periods or store closures. Furthermore, forecasting accuracy varried at different points of the horizon in particular during the future Christmas period and the endpoint of the horizon, this likely stems from the models difficulty in mapping Christmas peaks and dwindling data to predict further into the future. This evidences why a period of 13 weeks was chosen as a further forecast would likely bring greater uncertainty reducing the models forecasting accuracy. However, if additional data was available a forecasting for 6 months or 1 year may have been possible without experiencing decreases in forecast accuracy. Additionally, stores with more ambigous trends such as store 43 implied greater difficulty in accurate forecasts suggesting seasonality alone may not be the sole driver for some stores and would require further investigation to identify features which may impact weekly sales.

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

store_id = 1
store_df = Full_Walmart[Full_Walmart['Store'] == store_id].copy()

store_df.set_index('Date', inplace=True)
store_df.sort_index(inplace=True)
store_sales = store_df['Weekly_Sales']

decomposition = seasonal_decompose(store_sales, model='additive', period=52) 

decomposition.plot()
plt.suptitle(f"Time Series Decomposition for Store {store_id}", fontsize=16)
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

<p><b>Figure 15:</b> Time series decomposition for store 1.</p>

In [ ]:
store_id = 43
store_df = Full_Walmart[Full_Walmart['Store'] == store_id].copy()

store_df.set_index('Date', inplace=True)
store_df.sort_index(inplace=True)
store_sales = store_df['Weekly_Sales']

decomposition = seasonal_decompose(store_sales, model='additive', period=52) 

decomposition.plot()
plt.suptitle(f"Time Series Decomposition for Store {store_id}", fontsize=16)
plt.tight_layout()
plt.xticks(rotation=90)
plt.show()

<p><b>Figure 16:</b> Time series decomposition for store 43.</p>

To further understand forecasting accuracy STL (seasonal trend using LOESS) decompositions were produced. Overall, seasonal trend generally follows the original data from the examples. However residual clustering outside of the summer period, particularly for store 43, implied the presence of external factors aside from seasonality which impacted weekly sales. This in turn evidences the use of anomaly detection and strengthens the argument for further investigation into these periods or into specific stores. 

## Summary and Future Suggestions
Overall, the results of the forecasting gave insight into store trends and patterns where seasonality was a dominant factor in driving weekly  sales. However, there is evidence of additional features at play such as holidays in the case of the Christmas period and other unknown features, which warrant further investigation. Additionally, although all forecasts are highly accurate and most falling bellow a 6% error margin group 1 MAPE stores could still benefit from additional features being added to further imporve accuracy particularly for stores with unique trends.

However as evidenced, an investigation could take place to identify other drivers for weekly sales some suggestions include: promotion, internal store/supply issues and store closures to which, this information could improve the models capabilities and provide insight to determining the causes for anomalous points to reduce uncertainty. Additionally, some further non store related features could include: population demography, rural/urban locale (population counts), crime rates and median population income. Where these features could provide insights into weekly sales figures such as for example, differences could be expected in sale volumes between rural and urban locales as a result of population density. Additionally, this applies to median population income where typically areas with higher incomes would likely see more sales due to greater disposable income. 

As a final note, an additional use for this data type could be used to predict potential store performance for new stores. For example as used previously, dividing stores into performance groups, and using that as a replacement for weekly sales, could allow a classification algorithm to place new stores into performance categories based on a number of features. Where this method could provide insight when deciding between new store locations.  